In [ ]:
import torch
import numpy as np
import pandas as pd
import random # 시드 고정을 위해
import os # 시드 고정을 위해
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

모델 학습에 사용할 cpu or gpu 장치 확인

In [ ]:
SEED = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 재현성 함수(Reproduction)

In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)    # 파이썬 환경변수 시드 고정
    np.random.seed(seed)
    torch.manual_seed(seed) # cpu 연산 무작위 고정
    torch.cuda.manual_seed(seed) # gpu 연산 무작위 고정
    torch.backends.cudnn.deterministic = True  # cuda 라이브러리에서 Deterministic(결정론적)으로 예측하기 (예측에 대한 불확실성 제거 )

# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 구글 드라이브 데이터 경로(수정하시오)

In [ ]:
DATA_PATH = "/content/drive/MyDrive/0_강의/data/"

# 학습및 테스트 데이터

In [ ]:
train_churn = pd.read_csv(f"{DATA_PATH}train_churn.csv")
test_churn = pd.read_csv(f"{DATA_PATH}test_churn.csv")
train_churn.shape ,test_churn.shape

((5282, 21), (1761, 20))

# 결측치 확인

In [ ]:
train_churn.isnull().sum().sum() , test_churn.isnull().sum().sum()

(0, 0)

# 학습및 정답 데이터와 추론데이터 전처리

- 수치형 변수 

In [ ]:
train = train_churn.select_dtypes("number").iloc[:,:-1] # 정답값 제외
target = train_churn.iloc[:,-1] # 정답 데이터
test = test_churn.select_dtypes("number") # 추론 데이터

train.shape, test.shape , target.shape

((5282, 17), (1761, 17), (5282,))

- 범주형 변수 원핫 인코딩

In [ ]:
cols = ["InternetService","PaymentMethod"]
enc = OneHotEncoder(handle_unknown="ignore")

# 학습데이터
tmp = pd.DataFrame(
    enc.fit_transform(train_churn[cols]).toarray(),
    columns = enc.get_feature_names_out()
)

train = pd.concat([train,tmp],axis=1)

# 추론 데이터
tmp = pd.DataFrame(
    enc.transform(test_churn[cols]).toarray(),
    columns = enc.get_feature_names_out()
)

test = pd.concat([test,tmp],axis = 1)

train.shape, test.shape

((5282, 24), (1761, 24))

# 스케일링

In [ ]:
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

train.shape, test.shape , target.mean()

((5282, 24), (1761, 24), 0.2631578947368421)

In [ ]:
train

array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

# 여기서 부터 pytorch 를 이용하여  학습및 테스트 데이터에대한 확률을 추론해서 제출해 주세요.

In [ ]:
pred = 0 # 테스트 데이터에 대한 예측 확률, AUC 평가

# 제출파일에 예측확률 넣기

In [ ]:
submit = pd.read_csv(f"{DATA_PATH}submit_churn.csv")
submit["target"] = pred
submit

# 제출파일 구글드라이브에 저장

In [ ]:
submit.to_csv(f"{DATA_PATH}본인이름_submit.csv",index=False)